In [1]:
import pandas as pd
import numpy as np

dates=pd.date_range('2017',periods = 16, freq = 'Q')
sigma, mu = 1, 0
dataseries=pd.Series(sigma * np.random.randn(16) + mu,index=dates)
df=pd.DataFrame({'Test':dataseries})
df.head()

,Test
2017-03-31,-0.045148
2017-06-30,-2.238423
2017-09-30,-0.604435
2017-12-31,-0.847109
2018-03-31,0.620263


In [2]:
s1 = df['Test']

In [3]:
def calculate_ratio(ts1, ts2):
    '''
    '''
    return s1 / s2


def calculate_difference(ts):
    '''
    '''
    return ts - ts.shift(1)

def calculate_transform(ts):
    '''
    '''
    return ts / ts.shift(1) - 1

def calculate_inverse_difference(ts, start): 
    '''
    '''
    vals = [start]*len(ts)  # init fixed size array
    for i in range(1, len(ts)):
        vals[i] = vals[i-1] + ts.iloc[i]
    return pd.Series(vals, ts.index)

def calculate_inverse_transform(ts, start): 
    '''
    '''
    vals = [start]*len(ts)  # init fixed size array
    for i in range(1, len(ts)):
        vals[i] = vals[i-1] * (1 + ts.iloc[i])
    return pd.Series(vals, ts.index)


def calculate_standarization(ts, mu, sigma): 
    '''
    '''
    return (ts - mu) / sigma 

def calculate_real_interest_rate(rate_series, inflation_series, lag=1):
    """
    Computes the real (3 month) interest rate
    :param rate_series: Pandas time series1 3 month interest rates
    :param inflation_series: Pandas time series1 3 month interest rates
    :param lag: time lag (use 1 for yearly data, use 4 for quarterly data)
    :return: time series1 for real (3 month) interest rate
    """
    return rate_series - 100 * (inflation_series / inflation_series.shift(lag) - 1)

def _sample_to_yearly(series, aggregation_type):
    """
    Aggregate to yearly Q4 data.
    :param series: time series with quarterly data
    :param aggregation_type: 
    :return: time series with yearly (year-end) data
    """
    if aggregation_type == 'AVG':
        result = series.resample('A-DEC').mean()
        return result
    else:
        result = 4 * series.resample('A-DEC').mean()
        return result


def _sample_to_quarterly(series, aggregation_type):
    """
    Aggregate to yearly data to quarterly data.
    :param series: time series with yearly (year-end) data
    :param aggregation_type: 
    :return: time series with quarterly data
    """
    if aggregation_type == 'AVG':
        result = series.resample('Q').bfill()
        return result
    else:
        result = series.resample('Q').bfill() / 4
        return result


def _transform_yearly(series, transformation_type):
    """
    Transform to correct unit for extension.
    :param series: time series with yearly data
    :param transformation_type:
    :return: time series expressed in YOY (or Y-Y) unit
    """
    if transformation_type == 'YOY':
        result = calculate_transform(series)
        return result
    else:
        result = calculate_difference(series)
        return result

def _inverse_transform_yearly(time_series, transformation_type, start):
    """
    Transform from YOY (or Y-Y) to ABS unit.
    :param time_series: time series YOY (or Y-Y) unit
    :param transformation_type:
    :return: time series expressed in ABS unit
    """
    if transformation_type == 'YOY':
        result = calculate_inverse_transform(time_series, start)
        return result
    else:
        result = calculate_inverse_difference(time_series, start)
        return result

def extend_time_series(series, aggregation_type, transformation_type, end):
    """
    :param series: 
    :param aggregation_type: 
    :param transformation_type: 
    :param end: 
    :return: 
    """
    val0 = series.iloc[0]
    start = series.index[-1].year + 1
    
    index = pd.date_range(str(start), end, freq="A")
    

    transformed_series = _sample_to_yearly(series, aggregation_type)
    transformed_series = _transform_yearly(transformed_series, transformation_type)
    
    val1 = transformed_series[len(transformed_series)-1]
    val2 = transformed_series[len(transformed_series)-2]
    
    for time_stamp in index:  # start forecasting using index
        tmp = 0
        transformed_series[time_stamp] = tmp
        val2 = val1
        val1 = tmp
    print(transformed_series)
    extended_series = transformed_series[transformed_series.index.year >= start]
    extended_series = _inverse_transform_yearly(extended_series, transformation_type, val0)
    extended_series = _sample_to_quarterly(extended_series, aggregation_type)
    result = series.append(extended_series)
    
    return result

In [4]:
s3 = extend_time_series(s1, "AVG", "YOY", "2024-12-31")

2017-12-31         NaN
2018-12-31   -1.276242
2019-12-31    3.360258
2020-12-31   -1.060521
2021-12-31    0.000000
2022-12-31    0.000000
2023-12-31    0.000000
2024-12-31    0.000000
Freq: A-DEC, Name: Test, dtype: float64


In [5]:
s3

2017-03-31   -0.045148
2017-06-30   -2.238423
2017-09-30   -0.604435
2017-12-31   -0.847109
2018-03-31    0.620263
2018-06-30   -0.477740
2018-09-30    0.495459
2018-12-31    0.393814
2019-03-31    0.853319
2019-06-30   -0.803102
2019-09-30    2.856356
2019-12-31    1.592323
2020-03-31    0.155420
2020-06-30    0.435142
2020-09-30   -0.152267
2020-12-31   -0.710572
2021-12-31   -0.045148
2022-03-31   -0.045148
2022-06-30   -0.045148
2022-09-30   -0.045148
2022-12-31   -0.045148
2023-03-31   -0.045148
2023-06-30   -0.045148
2023-09-30   -0.045148
2023-12-31   -0.045148
2024-03-31   -0.045148
2024-06-30   -0.045148
2024-09-30   -0.045148
2024-12-31   -0.045148
dtype: float64

In [6]:
dates=pd.date_range('2017',periods = 16, freq = 'Q')
sigma, mu = 1, 0
dataseries=pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],index=dates)
df=pd.DataFrame({'Test':dataseries})
df['Type'] = [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
df

,Test,Type
2017-03-31,1,0
2017-06-30,2,0
2017-09-30,3,0
2017-12-31,4,0
2018-03-31,5,1
2018-06-30,6,1
2018-09-30,7,1
2018-12-31,8,1
2019-03-31,9,1
2019-06-30,10,1


In [7]:
# Q4 base:
series = df['Test']
s = series.resample('A-DEC').sum()[0:]
s

2017-12-31    10
2018-12-31    26
2019-12-31    42
2020-12-31    58
Freq: A-DEC, Name: Test, dtype: int64

In [8]:
# Q4 base:
series = df['Test']
s = series.resample('A-DEC').mean()[0:]
s

2017-12-31     2.5
2018-12-31     6.5
2019-12-31    10.5
2020-12-31    14.5
Freq: A-DEC, Name: Test, dtype: float64

In [9]:
# Q1 base:
series = df['Test']
s = series.resample('A-MAR').sum()[1:-1]
s

2018-03-31    14
2019-03-31    30
2020-03-31    46
Freq: A-MAR, Name: Test, dtype: int64

In [10]:
# Q4 base:
s0 = df['Test']
dates = series.index
s1  = series.resample('A-DEC').mean()
s1

2017-12-31     2.5
2018-12-31     6.5
2019-12-31    10.5
2020-12-31    14.5
Freq: A-DEC, Name: Test, dtype: float64

In [11]:
s2 = s1.resample('Q').bfill()
s2 = s2[s2.index.year >= 2018]
s2

2018-03-31     6.5
2018-06-30     6.5
2018-09-30     6.5
2018-12-31     6.5
2019-03-31    10.5
2019-06-30    10.5
2019-09-30    10.5
2019-12-31    10.5
2020-03-31    14.5
2020-06-30    14.5
2020-09-30    14.5
2020-12-31    14.5
Freq: Q-DEC, Name: Test, dtype: float64

In [12]:
import pandas as pd
pd.date_range("2015", "2017", freq="A")

DatetimeIndex(['2015-12-31', '2016-12-31'], dtype='datetime64[ns]', freq='A-DEC')

In [13]:
s2[-1]

14.5

In [14]:
s2[0]

6.5

In [15]:
s2.index[-1].year

2020

In [16]:
index = pd.date_range(s2.index[-1].year, "2017-12-31", freq="A")
index

DatetimeIndex(['1970-12-31 00:00:00.000002', '1971-12-31 00:00:00.000002',
               '1972-12-31 00:00:00.000002', '1973-12-31 00:00:00.000002',
               '1974-12-31 00:00:00.000002', '1975-12-31 00:00:00.000002',
               '1976-12-31 00:00:00.000002', '1977-12-31 00:00:00.000002',
               '1978-12-31 00:00:00.000002', '1979-12-31 00:00:00.000002',
               '1980-12-31 00:00:00.000002', '1981-12-31 00:00:00.000002',
               '1982-12-31 00:00:00.000002', '1983-12-31 00:00:00.000002',
               '1984-12-31 00:00:00.000002', '1985-12-31 00:00:00.000002',
               '1986-12-31 00:00:00.000002', '1987-12-31 00:00:00.000002',
               '1988-12-31 00:00:00.000002', '1989-12-31 00:00:00.000002',
               '1990-12-31 00:00:00.000002', '1991-12-31 00:00:00.000002',
               '1992-12-31 00:00:00.000002', '1993-12-31 00:00:00.000002',
               '1994-12-31 00:00:00.000002', '1995-12-31 00:00:00.000002',
               '1996-12-3

In [17]:
s2

2018-03-31     6.5
2018-06-30     6.5
2018-09-30     6.5
2018-12-31     6.5
2019-03-31    10.5
2019-06-30    10.5
2019-09-30    10.5
2019-12-31    10.5
2020-03-31    14.5
2020-06-30    14.5
2020-09-30    14.5
2020-12-31    14.5
Freq: Q-DEC, Name: Test, dtype: float64

In [18]:
s3 = s2[s2.index.year > 2019]

In [19]:
s3

2020-03-31    14.5
2020-06-30    14.5
2020-09-30    14.5
2020-12-31    14.5
Freq: Q-DEC, Name: Test, dtype: float64

In [20]:
result = s2.append(s3)

In [21]:
result

2018-03-31     6.5
2018-06-30     6.5
2018-09-30     6.5
2018-12-31     6.5
2019-03-31    10.5
2019-06-30    10.5
2019-09-30    10.5
2019-12-31    10.5
2020-03-31    14.5
2020-06-30    14.5
2020-09-30    14.5
2020-12-31    14.5
2020-03-31    14.5
2020-06-30    14.5
2020-09-30    14.5
2020-12-31    14.5
Name: Test, dtype: float64

In [22]:
s2

2018-03-31     6.5
2018-06-30     6.5
2018-09-30     6.5
2018-12-31     6.5
2019-03-31    10.5
2019-06-30    10.5
2019-09-30    10.5
2019-12-31    10.5
2020-03-31    14.5
2020-06-30    14.5
2020-09-30    14.5
2020-12-31    14.5
Freq: Q-DEC, Name: Test, dtype: float64

In [23]:
dates=pd.date_range('2017-06-30',periods = 16, freq = 'Q')
dataseries1=pd.Series([10, 10, 10, 20, 20, 20, 20, 30, 30, 30, 30, 40, 40, 40, 40, 50],index=dates)
dataseries1

2017-06-30    10
2017-09-30    10
2017-12-31    10
2018-03-31    20
2018-06-30    20
2018-09-30    20
2018-12-31    20
2019-03-31    30
2019-06-30    30
2019-09-30    30
2019-12-31    30
2020-03-31    40
2020-06-30    40
2020-09-30    40
2020-12-31    40
2021-03-31    50
Freq: Q-DEC, dtype: int64

In [24]:
dataseries2 = _sample_to_yearly(dataseries1, 'SUM')
dataseries2

2017-12-31     40
2018-12-31     80
2019-12-31    120
2020-12-31    160
2021-12-31    200
Freq: A-DEC, dtype: int64

In [25]:
INDEX=pd.date_range('2017',periods = 20, freq = 'Q')
lst = [dataseries1[idx] if idx in dataseries1.index else "NaN" for idx in INDEX]
lst = ["test1", "test2"] + lst
lst

['test1',
 'test2',
 'NaN',
 10,
 10,
 10,
 20,
 20,
 20,
 20,
 30,
 30,
 30,
 30,
 40,
 40,
 40,
 40,
 50,
 'NaN',
 'NaN',
 'NaN']

In [26]:
dates1=pd.date_range('2016',periods = 18, freq = 'A')
dates2=pd.date_range('2018',periods = 16, freq = 'A')
sigma, mu = 1, 0
dataseries1=pd.Series(sigma * np.random.randn(len(dates1)) + mu,index=dates1)
dataseries2=pd.Series(sigma * np.random.randn(len(dates2)) + mu,index=dates2)

In [27]:
test = calculate_real_interest_rate(dataseries1, dataseries2)
test

2016-12-31             NaN
2017-12-31             NaN
2018-12-31             NaN
2019-12-31      257.175358
2020-12-31       19.558216
2021-12-31      -67.393564
2022-12-31      107.840325
2023-12-31     2404.619993
2024-12-31      120.796156
2025-12-31      161.351475
2026-12-31      308.682749
2027-12-31      236.393159
2028-12-31      191.515529
2029-12-31      182.824666
2030-12-31      -57.182775
2031-12-31      101.767457
2032-12-31    10311.029215
2033-12-31       76.100144
Freq: A-DEC, dtype: float64

In [28]:
test.dropna()

2019-12-31      257.175358
2020-12-31       19.558216
2021-12-31      -67.393564
2022-12-31      107.840325
2023-12-31     2404.619993
2024-12-31      120.796156
2025-12-31      161.351475
2026-12-31      308.682749
2027-12-31      236.393159
2028-12-31      191.515529
2029-12-31      182.824666
2030-12-31      -57.182775
2031-12-31      101.767457
2032-12-31    10311.029215
2033-12-31       76.100144
Freq: A-DEC, dtype: float64

In [29]:
# date formating:
for idx in index:
   y = str(idx.to_datetime().date())
y

from datetime import datetime
t = 20070530
x = str(pd.to_datetime(str(t), format='%Y%m%d').to_datetime().date())
x

'2007-05-30'